In [3]:
import requests
import pandas as pd
import yaml
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import ADXIndicator, EMAIndicator, MACD


def get_real_time_stock(api_key: str, interval: str, stock_symbol: str) -> pd.DataFrame:
    response_values: str = f"Time Series ({interval})"
    alpha_vantage_url: str = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={stock_symbol}&interval={interval}&outputsize=full&extended_hours=false&apikey={api_key}"
    response = requests.get(alpha_vantage_url)
    real_time_stock_data = response.json().get(response_values)
    real_time_stock_dataframe: pd.DataFrame = pd.DataFrame(real_time_stock_data).transpose().sort_index().astype(float)

    new_column_names: dict = {
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume",
        "index": "datetime"

    }
    real_time_stock_dataframe = real_time_stock_dataframe.reset_index()
    real_time_stock_dataframe.rename(columns=new_column_names, inplace=True)

    bollinger_period: int = 20
    bollinger_std: int = 2
    macd_short_period: int = 12
    macd_long_period: int = 26
    macd_signal_period: int = 9
    adx_period: int = 14
    ema_period: int = 9
    rsi_period: int = 14

    bb = BollingerBands(close=real_time_stock_dataframe['close'], window=bollinger_period, window_dev=bollinger_std, fillna=True)
    macd = MACD(close=real_time_stock_dataframe['close'], window_fast=macd_short_period, window_slow=macd_long_period, window_sign=macd_signal_period, fillna=True)
    adx = ADXIndicator(high=real_time_stock_dataframe['high'], low=real_time_stock_dataframe['low'], close=real_time_stock_dataframe['close'], window=adx_period, fillna=True)
    ema = EMAIndicator(close=real_time_stock_dataframe['close'], window=ema_period)
    rsi = RSIIndicator(real_time_stock_dataframe['close'], window=rsi_period)

    real_time_stock_dataframe['percent_b'] = (real_time_stock_dataframe['close'] - bb.bollinger_lband()) / (bb.bollinger_hband() - bb.bollinger_lband())
    real_time_stock_dataframe['macd'] = macd.macd()
    real_time_stock_dataframe['macd_signal'] = macd.macd_signal()
    real_time_stock_dataframe['macd_hist'] = macd.macd_diff()
    real_time_stock_dataframe['adx'] = adx.adx()
    real_time_stock_dataframe['ema'] = ema.ema_indicator()
    real_time_stock_dataframe['rsi'] = rsi.rsi()
    print(real_time_stock_dataframe.shape)
    if response.status_code != 200:
        print(f"Error: {response.status_code} - {response.text}")
    else:
        return real_time_stock_dataframe


with open('../config.yaml', 'r') as config_file:
    config = yaml.safe_load(config_file)
alpha_api_key: str = config['alpha_vantage']['api_key']
alpha_stock_symbol: str = "AAPL"
alpha_interval: str = "5min"
test = get_real_time_stock(alpha_api_key, alpha_interval, alpha_stock_symbol)
test

(1092, 13)


,datetime,open,high,low,close,volume,percent_b,macd,macd_signal,macd_hist,adx,ema,rsi
0,2023-12-19 09:30:00,196.160,196.590,195.980,196.540,1978031.0,NaN,0.000000,0.000000,0.000000,0.000000,NaN,NaN
1,2023-12-19 09:35:00,196.550,196.600,196.000,196.310,771714.0,0.250000,-0.018348,-0.003670,-0.014678,0.000000,NaN,NaN
2,2023-12-19 09:40:00,196.312,196.655,196.120,196.460,709778.0,0.561187,-0.020548,-0.007045,-0.013502,0.000000,NaN,NaN
3,2023-12-19 09:45:00,196.470,196.500,195.890,196.040,654288.0,0.109733,-0.055541,-0.016744,-0.038797,0.000000,NaN,NaN
4,2023-12-19 09:50:00,196.040,196.560,195.920,196.534,621201.0,0.709372,-0.042918,-0.021979,-0.020939,0.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1087,2024-01-09 15:35:00,184.760,184.840,184.685,184.735,341323.0,0.795321,0.078694,0.060948,0.017746,9.409668,184.688483,56.515630
1088,2024-01-09 15:40:00,184.740,184.770,184.540,184.640,469536.0,0.603226,0.070987,0.062956,0.008031,9.388315,184.678786,52.325912
1089,2024-01-09 15:45:00,184.640,184.700,184.570,184.595,385005.0,0.513759,0.060550,0.062474,-0.001924,9.368487,184.662029,50.419197
1090,2024-01-09 15:50:00,184.600,185.020,184.410,184.970,1138023.0,1.111610,0.081597,0.066299,0.015298,10.247244,184.723623,62.637445
